In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet') 
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')


lowercase the columns names

In [3]:
data = [df_jan, df_feb]
for df in data:
    df.columns = df.columns.str.replace(' ', '_').lower()

AttributeError: 'Index' object has no attribute 'lower'

Merge the data of column february and january

In [ ]:
df = pd.concat([df_jan, df_feb], ignore_index=True) # merge 
data.append(df) # In case if we wanted to apply changes on each dataframe alone.
df.head(5)

In [ ]:
print('How many columns are there?')
print(df.shape[-1])

**Creating a duration vriable** Contain the duration of ride in minutes

In [ ]:
def get_duration(df):
    """
    calculate the duration of the trip
    
    (drop date - pickup date) / convert the diffrence to minutes representations using pd.Timedelta
    
    """
    
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)
    return df

df = get_duration(df)
df.head(2)

In [ ]:
print('the standard deviation of the trips duration in January?')
df_jan = get_duration(df_jan)
round(df_jan.duration.std(), 2)

**Checking the destribution of the data**

In [ ]:
def plot_duration(df):
    plt.figure(figsize=(10, 6))
    sns.histplot(df['duration'], bins=30, kde=True)
    plt.show()
plot_duration(df)

Looks like we have an extreme outliers. Let's detect them.

In [ ]:
def describe_duration(df):
    """
    Applying simple pd.describe at duration
    percentiles parameter by default will print(25%, 50%, 75%) but we can add more!
    .apply(lambda x:f'{x:.2f}') --> this format the output from describe (5.980721e+06 -> 15.84) to be human readable
    
    better way you can use -> pd.set_option('display.float_format', lambda x: '%.2f' % x)
    """
    print(df.duration.describe(percentiles=[.25, .75, .90, .95, .99]).apply(lambda x:f'{x:.2f}'))
describe_duration(df)

Now we can filter the data by removing the values above .99 percentile, below 0.1 to exclude the negatives and extremly high durations

In [ ]:
p_99 = df['duration'].quantile(.99)
p_01 = df['duration'].quantile(.01)
df_filtered = df[(p_01 < df['duration']) & (p_99 > df['duration'])]
describe_duration(df_filtered)

Sounds good! Let's check plot the distribution of df_filtered


In [ ]:
plot_duration(df_filtered)

now we can confidently go throught the modeling part\
I will use linear regression for simplicity

In [ ]:
df_filtered